**<font size=5 color=black>AdaBoost</font>**

**1** 介绍AdaBoost算法二分类的基本原理

**2** 介绍处理多分类问题的AdaBoost拓展算法SAMME

**3** 手动实现AdaBoost算法及其拓展算法SAMME

**4** 分别以简单二分类问题和鸢尾花多分类问题为例，对比本文算法与sklearn的结果

In [1]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
import numpy as np

**<font size=4 color=black>1 AdaBoost算法</font>**

**<font size=3.5 color=black>1.1 思想</font>**

给定一个训练样本集，首先利用一个弱学习器（如深度为1的简单决策树）来训练；然后调整训练样本集的样本权重，再次

利用弱学习器（可以与之前的一样，也可以不一样，重点在于”弱“，也就是要简单）来训练；最终，将所有的弱学习器

按一定规则进行组合，得到集成学习器，即完成我们的目标。

**在上述过程中，AdaBoost需要解决两个主要问题：**

**1）**每一轮的样本权重如何确定？

**2）**按什么规则组合弱分类器？

**<font size=3.5 color=black>1.2 流程</font>**

这里以二分类问题为例，给出AdaBoost算法的处理流程，并回答1.1中的两个问题。
$\rule[1pt]{23cm}{0.1em}$
**输入**：训练数据集$D={(x_1, y_1),(x_2, y_2),...,(x_N, y_N)},x_i \in R^n,y_i=\{-1, 1\}$；<br>
**输出**：分类器$G(x)$

**（1）**初始化训练数据的权值分布
$w_1=(w_{1,1},w_{1,2},...,w_{1,N}),w_{1,i}=\frac{1}{N},i=1,2,...,N\tag{1}$

**（2）**假设进行$m$轮训练，$m=1,2,...,M$

**（3）**训练权值为$w_m$的数据集，得到弱分类器（或者叫基本分类器）
$$G_m(x):x\to\{-1, 1\}\tag{2}$$

**（4）**计算$G_m(x)$在训练数据集的分类误差率
$$e_m=\sum_{i=1}^{N}w_{mi}I(G_m(x_i) \neq y_i)\tag{3}$$
可以理解为分类错误的样本的权值之和。

**（5）**计算$G_m(x)$的系数
$$\alpha_m=\frac{1}{2}ln\frac{1-e_m}{e_m}\tag{4}$$

**（6）**更新样本权值
$$w_{m+1,i}=(w_{m+1,1},w_{m+1,2},...,w_{m+1,N})\tag{5}$$
$$w_{m+1,i}=\frac{w_{m,i}}{Z_m}exp(-\alpha_m y_i G_m(x_i))\tag{6}$$
$$Z_m=\sum_{i=1}^{N}w_{m,i}exp(-\alpha_m y_i G_m(x_i))\tag{7}$$
这里引入$Z_m$的作用是对权值做归一化。

**（7）**组合弱分类器（线性组合）
$$f(x)=\sum_{m=1}^{M}\alpha_m G_m(x)\tag{8}$$

**（8）**最终分类器
$$G(x)=sign(f(x)=sign(\sum_{m=1}^{M}\alpha_m G_m(x))\tag{9}$$
$\rule[1pt]{23cm}{0.1em}$
**<font size=3.5 color=red>几点说明：</font>**

**（1）**初始的每个样本的权值是相等的，相当于每个样本在基本分类器学习中的作用相同，也就保证第一轮是在原始数据上学习$G(x_1)$；\
**（2）**公式(6)可以进一步写成：
$$ w_{m+1,i}=\left\{
\begin{aligned}
\frac{w_{m,i}}{Z_m}e^{-\alpha_m},G_m(x_i)=y_i \\
\frac{w_{m,i}}{Z_m}e^{\alpha_m},G_m(x_i) \neq y_i 
\end{aligned}
\right.
\tag{10}$$\
可以看出，AdaBoost每一轮对样本权值的更新是放大分类错误的样本权值，缩小分类正确的样本权值，以使得误分类样本

在下一轮的学习中起更大的作用。

**（3）**AdaBoost对基本分类器的组合方式是线性组合。由公式(4)可以看出，基本分类器的分类误差率$e_m$越小，$G_m(x)$的系数$\alpha_m$越大，即在组合时对于分类效果好的基本分类器给予更大的权重。

**（4）**AdaBoost主要处理二分类问题，一般不用于多分类问题。针对多分类问题更多的采用SAMME或SAMME.R算法，

详细可以参考[Multi-class AdaBoost](https://www.intlpress.com/site/pub/pages/journals/items/sii/content/vols/0002/0003/a008/)。

**<font size=3.5 color=black>1.3 相关证明</font>**

AdaBoost算法可以解释为模型为加法模型、损失函数为指数函数、学习算法为前向分布算法的二类分类学习方法。

详细证明见[1]。

**<font size=4 color=black>2 SAMME算法</font>**

SAMME算法的提出是为了解决AdaBoost算法难以处理多分类任务这一问题，其基本思想与AdaBoost算法相同，只是在

(4)、(6)、(8)、(9)公式上有一些差异，下面给出相关公式细节。

**<font size=3.5 color=black>2.1 SAMME算法中的几个重要公式</font>**

**（1）**$G_m(x)$的系数
$$\alpha_m=\frac{1}{2}ln\frac{1-e_m}{e_m}+ln(K-1)\tag{11}$$
$K$为训练数据集中样本的类别个数，当$K=2$时，等价于公式(4)。

**（2）**样本权值更新
$$w_{m+1,i}=\frac{w_{m,i}}{Z_m}exp(\alpha_m I(y_{pred} \neq y_{truth}))\tag{12}$$
$$Z_m=\sum_{i=1}^{N}w_{m,i}exp(\alpha_m I(y_{pred} \neq y_{truth}))\tag{13}$$

**（3）**预测
$$G(x)=\mathop {argmax} \limits_{k} \sum_{m=1}^{M}\alpha_m I(G_m(x)=k)\tag{14}$$

**<font size=3.5 color=black>2.2 SAMME算法和AdaBoost算法的代码说明</font>**

**（1）**为便于理解，本文在编写代码过程中尽可能按照给出的公式来实现，因此应用AdaBoost算法需要预先对二分类数据的标签设置为$\{-1, 1\}$，但需要清楚实际这并不是一项必要的工作，可以改写部分代码来处理这一问题。

**（2）**SAMME算法也可以进行二分类。因此在调用时请注意：二分类问题可以选择算法有AdaBoost和SAMME，多分类问

题仅可选择SAMME。

**（3）**由于本文重点研究AdaBoost算法的学习流程，对于基本分类器的实现借助于[sklearn.tree.DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)，

可以参考本项目的**DecisionTree.ipynb**部分了解更多。

**（4）**本文仅限于帮助学习相关算法的基础理论，若想要更深入地学习，可以参考[sklearn.ensemble.AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)

及实现源码。

**<font size=4 color=black>3 手动实现AdaBoost算法和SAMME算法</font>**

In [2]:
class MyAdaBoostClassifier(object):
    """AdaBoost分类器   
    Parameters
    ----------
    base_estimator: class 
        基分类器 默认为DecisionTreeClassifier(max_depth=1, random_state=42)
    n_estimators: int 
        基分类器的个数
    learning_rate: float
        用于调整每个基分类器的权重，其值越大，基分类器的作用越大，默认值为1
    algorithm: 
        {'AdaBoost','SAMME'} 默认'SAMME'

    Attributes
    ----------
    classes_: ndarray of shape (n_classes_,)
        数据集中的样本类别
    n_classes_: int
        数据集中的样本类别的个数
    estimators_: list of classifiers
        每一轮训练的基分类器
    estimator_weights_: ndarray of shape (n_estimators_,)
        基分类器的权值
    estimator_errors_: ndarray of shape (n_estimators_,)
        基分类器的分类误差率
    """

    def __init__(self, base_estimator=DecisionTreeClassifier,
                 n_estimators=3, learning_rate=1.0, algorithm='SAMME'):
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.algorithm = algorithm
        self.classes_ = None
        self.n_classes_ = None
        self.estimators_ = []
        self.estimator_weights_ = []
        self.estimator_errors_ = []

    def fit(self, X, y):
        """训练分类器
        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features) 样本向量
        y: ndarray of shape (n_samples,) 样本标签
        """
        self.classes_ = np.unique(y)
        self.n_classes_ = len(self.classes_)

        # 初始化样本权值
        w = np.array([1 / len(y)] * len(y))
        for i in range(self.n_estimators):
            # 初始化基分类器
            estimator = self.base_estimator(max_depth=1, random_state=42)
            # 训练基分类器
            estimator.fit(X, y, sample_weight=w)
            # 基分类器预测
            y_pre = estimator.predict(X)
            # 计算基分类器的分类误差率
            e = np.sum((y_pre != y) * w)

            # 计算基分类器权值并更新样本权值
            if self.algorithm == 'SAMME':
                alpha = self.learning_rate * np.log((1-e) / e) + \
                    np.log(self.n_classes_-1)
                w = w * np.exp(alpha * (y_pre != y))
                w = w / np.sum(w)
            elif self.algorithm == 'AdaBoost':
                alpha = 0.5 * np.log((1-e) / e)
                w = w * np.exp(-alpha * y * y_pre)
                w = w / np.sum(w)

            self.estimators_.append(estimator)
            self.estimator_weights_.append(alpha)
            self.estimator_errors_.append(e)

    def predict(self, X):
        """预测分类标签
        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features) 样本向量

        Returns
        -------
        out: ndarray of shape (n_samples,) 分类标签
        """
        classes_ = self.classes_[:, np.newaxis]
        n_classes_ = self.n_classes_

        # 集成分类器预测
        if n_classes_ == 2:
            pred = np.zeros(X.shape[0])
            for alpha, estimator in zip(self.estimator_weights_, self.estimators_):
                pred += alpha * estimator.predict(X)
            return np.sign(pred)
        else:
            pred = sum((estimator.predict(X) == classes_).T * alpha
                       for estimator, alpha in zip(self.estimators_, self.estimator_weights_))
            # 这里参考sklearn进行归一化，上文公式未给出，但不影响输出结果
            pred /= np.sum(self.estimator_weights_)  
            return self.classes_.take(np.argmax(pred, axis=1), axis=0)

**<font size=4 colot=black>3 案例</font>**

**<font size=3.5 colot=black>3.1 二分类</font>**

In [3]:
# 数据准备
X1 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]).reshape(-1, 1)
y1 = np.array([1, 1, 1, -1, -1, -1, 1, 1, 1, -1])

In [4]:
# 本文算法-AdaBoost
myada = MyAdaBoostClassifier(n_estimators=3, algorithm='AdaBoost')
myada.fit(X1, y1)

print('本文算法-AdaBoost')
print('-----------------')
print('集成分类器预测结果：\n', myada.predict(X1))
print('子分类器权重：\n', myada.estimator_weights_)
print('子分类器误差率：\n', myada.estimator_errors_)

本文算法-AdaBoost
-----------------
集成分类器预测结果：
 [ 1.  1.  1. -1. -1. -1.  1.  1.  1. -1.]
子分类器权重：
 [0.4236489301936017, 0.6496414920651304, 0.752038698388137]
子分类器误差率：
 [0.30000000000000004, 0.21428571428571427, 0.18181818181818185]


可以看出使用AdaBoost算法得到的结果与文献[1]中所对应的例子的结果一致。

In [5]:
# 本文算法-SAMME
myada = MyAdaBoostClassifier(n_estimators=3, algorithm='SAMME')
myada.fit(X1, y1)

print('本文算法-SAMME')
print('-------------')
print('集成分类器预测结果：\n', myada.predict(X1))
print('子分类器权重：\n', myada.estimator_weights_)
print('子分类器误差率：\n', myada.estimator_errors_)

本文算法-SAMME
-------------
集成分类器预测结果：
 [ 1.  1.  1. -1. -1. -1.  1.  1.  1. -1.]
子分类器权重：
 [0.8472978603872034, 1.2992829841302609, 1.504077396776274]
子分类器误差率：
 [0.30000000000000004, 0.21428571428571427, 0.18181818181818182]


In [6]:
# sklearn
ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, random_state=42),
                         n_estimators=3, algorithm='SAMME')
ada.fit(X1, y1)

print('sklearn')
print('-------')
print('集成分类器预测结果：\n', ada.predict(X1))
print('子分类器权重：\n', ada.estimator_weights_)
print('子分类器误差率：\n', ada.estimator_errors_)

sklearn
-------
集成分类器预测结果：
 [ 1  1  1 -1 -1 -1  1  1  1 -1]
子分类器权重：
 [0.84729786 1.29928298 1.5040774 ]
子分类器误差率：
 [0.3        0.21428571 0.18181818]


二分类实例的本文算法与sklearn得到的结果一致。但sklearn的实现细节与本文有所差异，读者可以自行查阅源码了解。

**<font size=3.5 colot=black>3.2 多分类</font>**

In [7]:
iris = load_iris()
X2 = iris.data
y2 = iris.target

In [8]:
# 本文算法
myada = MyAdaBoostClassifier(n_estimators=3, algorithm='SAMME')
myada.fit(X2, y2)

print('本文算法')
print('-------')
print('集成分类器预测结果：\n', myada.predict(X2[[0, 50, 100], :]))
print('子分类器权重：\n', myada.estimator_weights_)
print('子分类器误差率：\n', myada.estimator_errors_)

本文算法
-------
集成分类器预测结果：
 [0 1 2]
子分类器权重：
 [1.386294361119891, 2.2094946699280333, 2.742455876638902]
子分类器误差率：
 [0.33333333333333326, 0.18000000000000002, 0.11412225233363443]


In [9]:
# sklearn
ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, random_state=42),
                         n_estimators=3, algorithm='SAMME')
ada.fit(X2, y2)

print('sklearn')
print('-------')
print('集成分类器预测结果：\n', ada.predict(X2[[0, 50, 100], :]))
print('子分类器权重：\n', ada.estimator_weights_)
print('子分类器误差率：\n', ada.estimator_errors_)

sklearn
-------
集成分类器预测结果：
 [0 1 2]
子分类器权重：
 [1.38629436 2.20949467 2.74245588]
子分类器误差率：
 [0.33333333 0.18       0.11412225]


多分类实例的本文算法与sklearn的结果一致。在该部分的代码实现过程中，参考了sklearn对应源码的部分实现细节。

**<font color=black size=4>参考</font>**

1 李航. (2012) 统计学习方法. 清华大学出版社, 北京.

2 [sklearn.ensemble.AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)